<a href="https://colab.research.google.com/github/AntonioCuba123/skills-introduction-to-github/blob/main/Modelo_de_predicci%C3%B3n_de_abandono_de_membres%C3%ADas_en_restauraci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importamos las bibliotecas necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

# Simulamos un conjunto de datos para el ejemplo
np.random.seed(42)

# Generamos 1000 clientes de ejemplo
n_samples = 1000

# Creamos características con distribuciones realistas
data = {
    # Demográficos
    'edad': np.random.normal(38, 12, n_samples).astype(int),
    'distancia_km': np.random.exponential(5, n_samples),
    'nivel_ingresos': np.random.choice(['bajo', 'medio', 'alto', 'premium'], n_samples,
                                      p=[0.2, 0.45, 0.25, 0.1]),

    # Comportamiento
    'visitas_mensuales': np.random.poisson(2.5, n_samples),
    'consumo_promedio': np.random.normal(35, 15, n_samples),
    'dias_ultima_visita': np.random.exponential(30, n_samples).astype(int),
    'variedad_platos': np.random.poisson(3, n_samples),

    # Programa
    'meses_antiguedad': np.random.gamma(shape=5, scale=3, size=n_samples).astype(int),
    'uso_beneficios': np.random.beta(2, 5, n_samples),  # Porcentaje de beneficios usados
    'eventos_asistidos': np.random.poisson(1, n_samples),
    'promociones_redimidas': np.random.poisson(3, n_samples),

    # Satisfacción
    'calificacion': np.random.normal(4, 1, n_samples).clip(1, 5),
    'quejas_presentadas': np.random.poisson(0.3, n_samples),
    'problemas_resueltos': np.random.binomial(1, 0.8, n_samples)  # 1 si el problema fue resuelto
}

# Convertimos a DataFrame
df = pd.DataFrame(data)

# Generamos una variable objetivo sintética basada en patrones realistas
# Los clientes con mayor probabilidad de abandono suelen tener:
# - Baja frecuencia de visitas
# - Más días desde la última visita
# - Menos uso de beneficios
# - Más quejas y menor calificación

# Calculamos un puntaje de riesgo
risk_score = (
    -0.3 * df['visitas_mensuales'] +
    0.02 * df['dias_ultima_visita'] +
    -0.5 * df['uso_beneficios'] +
    0.7 * df['quejas_presentadas'] +
    -0.4 * df['calificacion'] +
    -0.1 * df['meses_antiguedad'] +
    0.2 * (df['problemas_resueltos'] == 0)
)

# Normalizamos y convertimos a probabilidad con sigmoide
probability = 1 / (1 + np.exp(-risk_score))

# Asignamos la clase (abandono=1) con cierta aleatoriedad
df['abandono'] = (probability > np.random.uniform(0.3, 0.7, n_samples)).astype(int)

# Convertimos variables categóricas a numéricas
df = pd.get_dummies(df, columns=['nivel_ingresos'], drop_first=True)

# Dividimos en conjunto de entrenamiento y prueba
X = df.drop('abandono', axis=1)
y = df['abandono']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Estandarizamos las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenamos un modelo de Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Evaluamos el modelo
y_pred = model.predict(X_test_scaled)
y_prob = model.predict_proba(X_test_scaled)[:, 1]

# Calculamos métricas de rendimiento
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))
print("\nInforme de clasificación:")
print(classification_report(y_test, y_pred))
print(f"\nROC AUC Score: {roc_auc_score(y_test, y_prob):.4f}")

# Visualizamos la importancia de las características
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=feature_importance.head(10))
plt.title('Las 10 características más importantes')
plt.tight_layout()
plt.show()

# Implementamos un sistema de puntuación para identificar clientes en riesgo


def calcular_riesgo_cliente(datos_cliente):
    # Creamos un DataFrame con los datos del nuevo cliente
    # Usamos las columnas de X_train para asegurar la compatibilidad
    nuevo_cliente_df = pd.DataFrame([datos_cliente], columns=[
        'edad', 'distancia_km', 'visitas_mensuales', 'consumo_promedio',
        'dias_ultima_visita', 'variedad_platos', 'meses_antiguedad',
        'uso_beneficios', 'eventos_asistidos', 'promociones_redimidas',
        'calificacion', 'quejas_presentadas', 'problemas_resueltos',
        'nivel_ingresos_medio', 'nivel_ingresos_premium', 'nivel_ingresos_alto'
    ])
    # Get missing columns in nuevo_cliente_df compared to X_train
    missing_cols = set(X_train.columns) - set(nuevo_cliente_df.columns)

    # Add missing columns with 0 values to nuevo_cliente_df
    for col in missing_cols:
        nuevo_cliente_df[col] = 0

    # Reordenamos las columnas para que coincidan con X_train
    # Esto es importante para que el StandardScaler funcione correctamente
    # Solo seleccionamos las columnas que están presentes en X_train
    nuevo_cliente_df = nuevo_cliente_df[X_train.columns]

    # Ahora podemos escalar los datos del nuevo cliente
    datos_procesados = scaler.transform(nuevo_cliente_df)

    # Predecimos la probabilidad de abandono
    probabilidad_abandono = model.predict_proba(datos_procesados)[0, 1]

    # Asignamos el nivel de riesgo
    if probabilidad_abandono > 0.7:
        nivel_riesgo = "Alto"
        acciones = "Contacto prioritario, oferta personalizada de valor"
    elif probabilidad_abandono > 0.4:
        nivel_riesgo = "Medio"
        acciones = "Envío de promoción especial, recordatorio de beneficios"
    else:
        nivel_riesgo = "Bajo"
        acciones = "Mantener comunicación regular"

    return {
        "probabilidad": probabilidad_abandono,
        "nivel_riesgo": nivel_riesgo,
        "acciones_recomendadas": acciones
    }

# Ejemplo de uso con un cliente nuevo
# Asegúrate de incluir los valores para las variables one-hot encoded
# en el mismo orden en que se crearon durante el entrenamiento
nuevo_cliente = [35, 3.5, 2, 30, 7, 3, 10, 0.5, 1, 2, 4, 0, 1, 0, 0, 1] # Ejemplo: nivel_ingresos = alto
resultado = calcular_riesgo_cliente(nuevo_cliente)
print("\nEvaluación de riesgo del nuevo cliente:")
print(f"Probabilidad de abandono: {resultado['probabilidad']:.2f}")
print(f"Nivel de riesgo: {resultado['nivel_riesgo']}")
print(f"Acciones recomendadas: {resultado['acciones_recomendadas']}")